<a href="https://colab.research.google.com/github/rajaranjith/HCL-GenAI-Training/blob/main/fintech_fraud_investigation_report_agentic_ai_HCLAICafe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-dotenv
!pip install langchain
!pip install langgraph

In [2]:
!pip install langchain_openai
!pip install langchain_community langchain_chroma

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.8/85.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 13.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.9
    Uninstalling langchain-core-1.2.9:
      Successfully uninstalled langchain-core-1.2.9
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from dotenv import load_dotenv
import os
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, ToolMessage
from operator import add as add_messages
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
#from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_core.tools import tool

In [4]:
from google.colab import userdata
from dotenv import load_dotenv
import os
load_dotenv()
os.environ["AZURE_OPENAI_API_KEY"] = "a8296d0e-0312-429f-9fd7-729c4fdc4b12"
os.environ["AZURE_OPENAI_ENDPOINT"] = ("https://aicafe.hcl.com/AICafeService/api/v1/subscription/openai/deployments/gpt-4.1/chat/completions?api-version=2024-12-01-preview")
os.environ["OPENAI_API_VERSION"]="2024-12-01-preview"

# Get the API key from environment variables
#openai_api_key = userdata.get("OPENAI_API_KEY")
#import os
#os.environ["OPENAI_API_KEY"] = openai_api_key

from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings

llm = AzureChatOpenAI(
    azure_deployment="gpt-4.1",
    temperature=0
)

embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-ada-002",
    azure_endpoint="https://aicafe.hcl.com/AICafeService/api/v1/subscription/openai/deployments/ada/embeddings?api-version=2023-05-15",
    api_key="a8296d0e-0312-429f-9fd7-729c4fdc4b12"
)

In [5]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 21.0 MB/s eta 0:00:00


In [6]:
from typing import TypedDict, Dict, List
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langgraph.graph import StateGraph, END

llm = AzureChatOpenAI(
    azure_deployment="gpt-4.1",
    temperature=0
)

embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-ada-002",
    azure_endpoint="https://aicafe.hcl.com/AICafeService/api/v1/subscription/openai/deployments/ada/embeddings?api-version=2023-05-15",
    api_key="a8296d0e-0312-429f-9fd7-729c4fdc4b12"
)

# -------------------------------------------------------------------
# VECTOR DATABASE (FRAUD / AML KNOWLEDGE)
# -------------------------------------------------------------------

documents = [
    Document(page_content="Rapid high-value transactions may indicate account takeover."),
    Document(page_content="Transactions inconsistent with historical behavior are strong fraud indicators."),
    Document(page_content="Cross-border transfers above thresholds require enhanced AML checks."),
    Document(page_content="Multiple failed logins followed by a large transfer indicate fraud."),
    Document(page_content="Dormant accounts suddenly becoming active is a known laundering pattern.")
]

vectorstore = FAISS.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

In [7]:
# -------------------------------------------------------------------
# UPDATED SHARED STATE
# -------------------------------------------------------------------

class FraudState(TypedDict):
    transaction: Dict[str, float]
    user_profile: Dict[str, float]
    remaining_agents: List[str]

    behavior_assessment: str
    pattern_assessment: str
    compliance_assessment: str

    final_risk: str
    action: str


# -------------------------------------------------------------------
# ORCHESTRATOR (SEQUENTIAL PLANNER)
# -------------------------------------------------------------------

import json

def orchestrator_agent(state: FraudState):

    prompt = f"""
    You are a fraud investigation planner.

    Transaction:
    {state["transaction"]}

    User profile:
    {state["user_profile"]}

    Decide execution order using:
    - behavior
    - pattern
    - compliance

    Respond ONLY in JSON:

    {{
        "execution_order": ["behavior", "pattern", "compliance"]
    }}
    """

    response = llm.invoke(prompt).content.strip()

    try:
        parsed = json.loads(response)
        state["remaining_agents"] = parsed["execution_order"]
    except:
        state["remaining_agents"] = ["behavior", "pattern", "compliance"]

    return state

def first_router(state: FraudState):
    return state["remaining_agents"][0]


In [8]:
def behavior_agent(state: FraudState):
    state["behavior_assessment"] = llm.invoke(
        f"Assess behavior risk: {state['user_profile']}"
    ).content

    state["remaining_agents"].pop(0)
    return state


def pattern_agent(state: FraudState):
    retrieved = retriever.invoke("transaction anomaly")
    context = "\n".join(d.page_content for d in retrieved)

    state["pattern_assessment"] = llm.invoke(
        f"Assess pattern risk using context:\n{context}"
    ).content

    state["remaining_agents"].pop(0)
    return state

def compliance_agent(state: FraudState):
    state["compliance_assessment"] = llm.invoke(
        f"Assess AML risk: {state['transaction']}"
    ).content

    state["remaining_agents"].pop(0)
    return state

In [9]:
def judge_agent(state: FraudState):
    prompt = f"""
    You are a senior fraud risk judge.

    Behavior assessment:
    {state.get("behavior_assessment", "")}

    Pattern assessment:
    {state.get("pattern_assessment", "")}

    Compliance assessment:
    {state.get("compliance_assessment", "")}

    Decide overall fraud risk.

    Respond with exactly one word:
    LOW, MEDIUM, or HIGH
    """

    state["final_risk"] = llm.invoke(prompt).content.strip()
    return state

In [10]:
def action_agent(state: FraudState):

    risk = state.get("final_risk", "LOW")

    if "HIGH" in risk:
        state["action"] = "BLOCK_TRANSACTION"
    elif "MEDIUM" in risk:
        state["action"] = "ESCALATE_TO_HUMAN"
    else:
        state["action"] = "ALLOW_TRANSACTION"

    return state

In [11]:
def report_node(state: FraudState):

    print("\n========== FRAUD INVESTIGATION REPORT ==========")
    print("Transaction:", state["transaction"])
    print("User Profile:", state["user_profile"])
    print("\nBehavior Assessment:\n", state.get("behavior_assessment", "N/A"))
    print("\nPattern Assessment:\n", state.get("pattern_assessment", "N/A"))
    print("\nCompliance Assessment:\n", state.get("compliance_assessment", "N/A"))
    print("\nFINAL RISK:", state.get("final_risk", "UNKNOWN"))
    print("ACTION:", state.get("action", "NONE"))
    print("===============================================\n")

    return state

In [12]:
graph = StateGraph(FraudState)

graph.add_node("orchestrator", orchestrator_agent)
graph.add_node("behavior", behavior_agent)
graph.add_node("pattern", pattern_agent)
graph.add_node("compliance", compliance_agent)
graph.add_node("judge", judge_agent)
graph.add_node("action", action_agent)
graph.add_node("report", report_node)

graph.set_entry_point("orchestrator")

graph.add_conditional_edges(
    "orchestrator",
    first_router,
    {
        "behavior": "behavior",
        "pattern": "pattern",
        "compliance": "compliance"
    }
)

# Fixed sequential flow
graph.add_edge("behavior", "pattern")
graph.add_edge("pattern", "compliance")
graph.add_edge("compliance", "judge")
graph.add_edge("judge", "action")
graph.add_edge("action", "report")
graph.add_edge("report", END)

app = graph.compile()

In [13]:
app.invoke({
    "transaction": {
        "amount": 15000,
        "country_risk_score": 0.9,
        "hour": 2
    },
    "user_profile": {
        "account_age_years": 0.4,
        "avg_transaction": 300,
        "login_failures": 5
    },
    "remaining_agents": [],
    "behavior_assessment": "",
    "pattern_assessment": "",
    "compliance_assessment": "",
    "final_risk": "",
    "action": ""
})


========== FRAUD INVESTIGATION REPORT ==========
Transaction: {'amount': 15000, 'country_risk_score': 0.9, 'hour': 2}
User Profile: {'account_age_years': 0.4, 'avg_transaction': 300, 'login_failures': 5}

Behavior Assessment:
 Here's an assessment of the behavioral risk based on the provided data:

**Input Data:**
- Account age: 0.4 years (~5 months)
- Average transaction: $300
- Login failures: 5

**Risk Assessment:**

1. **Account Age (0.4 years):**
   - Newer accounts are generally considered higher risk, especially if they show unusual activity.
   - **Risk Level:** Moderate to High

2. **Average Transaction ($300):**
   - The risk depends on the typical transaction size for your platform. If $300 is significantly higher than average, this could be a red flag, especially for a new account.
   - **Risk Level:** Moderate (if $300 is above average for new users)

3. **Login Failures (5):**
   - Multiple failed login attempts can indicate account takeover attempts or user error. Five 

{'transaction': {'amount': 15000, 'country_risk_score': 0.9, 'hour': 2},
 'user_profile': {'account_age_years': 0.4,
  'avg_transaction': 300,
  'login_failures': 5},
 'remaining_agents': [],
 'behavior_assessment': "Here's an assessment of the behavioral risk based on the provided data:\n\n**Input Data:**\n- Account age: 0.4 years (~5 months)\n- Average transaction: $300\n- Login failures: 5\n\n**Risk Assessment:**\n\n1. **Account Age (0.4 years):**\n   - Newer accounts are generally considered higher risk, especially if they show unusual activity.\n   - **Risk Level:** Moderate to High\n\n2. **Average Transaction ($300):**\n   - The risk depends on the typical transaction size for your platform. If $300 is significantly higher than average, this could be a red flag, especially for a new account.\n   - **Risk Level:** Moderate (if $300 is above average for new users)\n\n3. **Login Failures (5):**\n   - Multiple failed login attempts can indicate account takeover attempts or user error